# <center>
<div style="
    background: #f0f333ff;
    border-left: 5px solid #ecd242ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h1 style="
        text-align: center;
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 600;
    ">
    My Scheduler
    </h1>
</div>

##
<div style="
    background: #40f0aa;
    border-left: 5px solid #0c7230ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h2 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    CA Scheduling
    </h2>
</div>

###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    First test : with chatGPT
    </h3>
</div>

In [39]:
from ortools.sat.python import cp_model

def main():
    # -------------------------
    # Paramètres du problème
    # -------------------------
    employees = ["A", "B", "C", "D"]
    days = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
    shifts = ["Matin", "Après-midi"]
    activities = ["Telephone", "Derogation", "Reclamation"]

    n_emp = len(employees)
    n_days = len(days)
    n_shifts = len(shifts)

    # -------------------------
    # Modèle
    # -------------------------
    model = cp_model.CpModel()

    # x[e, d, s, a] = 1 si l'employé e fait l'activité a le jour d et shift s
    x = {}
    for e in range(n_emp):
        for d in range(n_days):
            for s in range(n_shifts):
                for a in range(len(activities)):
                    x[e, d, s, a] = model.NewBoolVar(f"x_{employees[e]}_{days[d]}_{shifts[s]}_{activities[a]}")

    # -------------------------
    # Contraintes
    # -------------------------

    # 1️⃣ Chaque créneau : 2 Téléphones, 1 Dérogation, 1 Réclamation
    for d in range(n_days):
        for s in range(n_shifts):
    #         # Chaque activité a le bon nombre de personnes
            model.Add(sum(x[e, d, s, 0] for e in range(n_emp)) == 2)  # Téléphone
            model.Add(sum(x[e, d, s, 1] for e in range(n_emp)) == 1)  # Dérogation
            model.Add(sum(x[e, d, s, 2] for e in range(n_emp)) == 1)  # Réclamation

            # Chaque employé fait exactement 1 activité par créneau
            for e in range(n_emp):
                model.Add(sum(x[e, d, s, a] for a in range(len(activities))) == 1)

    # 2️⃣ Pas plus de 3 jours de Téléphone par semaine par employé
    # Un jour est compté Téléphone s'il fait téléphone matin ou après-midi
    for e in range(n_emp):
        is_tel_day = []
        for d in range(n_days):
            tel_day = model.NewBoolVar(f"tel_day_{employees[e]}_{days[d]}")
            model.Add(sum(x[e, d, s, 0] for s in range(n_shifts)) >= 1).OnlyEnforceIf(tel_day)
            model.Add(sum(x[e, d, s, 0] for s in range(n_shifts)) == 0).OnlyEnforceIf(tel_day.Not())
            is_tel_day.append(tel_day)
        model.Add(sum(is_tel_day) <= 3)

    # 3️⃣ Pas 3 jours consécutifs de Téléphone
    for e in range(n_emp):
        for d in range(n_days - 2):
            model.Add(sum(x[e, d + i, s, 0] for i in range(3) for s in range(n_shifts)) <= 4)
            # (au max 4 créneaux sur 6 possibles pour 3 jours → évite 3 jours pleins de téléphone)

    # -------------------------
    # Solveur
    # -------------------------
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10
    solver.parameters.num_search_workers = 8

    result = solver.Solve(model)

    # -------------------------
    # Affichage du planning
    # -------------------------
    if result == cp_model.OPTIMAL or result == cp_model.FEASIBLE:
        for d in range(n_days):
            print(f"\n=== {days[d]} ===")
            for s in range(n_shifts):
                print(f"  {shifts[s]} :")
                for e in range(n_emp):
                    for a in range(len(activities)):
                        if solver.Value(x[e, d, s, a]) == 1:
                            print(f"    {employees[e]} → {activities[a]}")
    else:
        print("❌ Aucune solution trouvée.")

if __name__ == "__main__":
    main()


=== Lundi ===
  Matin :
    A → Derogation
    B → Telephone
    C → Reclamation
    D → Telephone
  Après-midi :
    A → Derogation
    B → Telephone
    C → Reclamation
    D → Telephone

=== Mardi ===
  Matin :
    A → Derogation
    B → Reclamation
    C → Telephone
    D → Telephone
  Après-midi :
    A → Derogation
    B → Reclamation
    C → Telephone
    D → Telephone

=== Mercredi ===
  Matin :
    A → Telephone
    B → Telephone
    C → Derogation
    D → Reclamation
  Après-midi :
    A → Telephone
    B → Telephone
    C → Reclamation
    D → Derogation

=== Jeudi ===
  Matin :
    A → Derogation
    B → Reclamation
    C → Telephone
    D → Telephone
  Après-midi :
    A → Telephone
    B → Derogation
    C → Telephone
    D → Reclamation

=== Vendredi ===
  Matin :
    A → Telephone
    B → Telephone
    C → Reclamation
    D → Derogation
  Après-midi :
    A → Derogation
    B → Telephone
    C → Telephone
    D → Reclamation


**My custom**

Tu veux donc un planning hebdomadaire (5 jours) avec :

4 employés (nommons-les : A, B, C, D)

2 créneaux par jour (matin, après-midi)

3 activités : téléphone, dérogation, réclamation

Contraintes :

Chaque créneau (matin, après-midi) → 2 Téléphones, 1 Dérogation, 1 Réclamation

Un employé ne fait pas plus de 3 jours de Téléphone par semaine.

Un employé ne fait pas 3 jours consécutifs de Téléphone (le matin et/ou l’après-midi comptent comme « du téléphone »).

In [59]:
from ortools.sat.python import cp_model

def main():
    # -------------------------
    # Paramètres du problème
    # -------------------------
    employees = ["A", "B", "C", "D"]
    days = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
    shifts = ["Matin", "Après-midi"]
    activities = ["Tél", "Dérog", "Récla"]

    n_emp = len(employees)
    n_days = len(days)
    n_shifts = len(shifts)

    # -------------------------
    # Modèle
    # -------------------------
    model = cp_model.CpModel()

    # x[e, d, s, a] = 1 si l'employé e fait l'activité a le jour d et shift s
    x = {}
    for e in range(n_emp):
        for d in range(n_days):
            for s in range(n_shifts):
                for a in range(len(activities)):
                    x[e, d, s, a] = model.NewBoolVar(f"x_{employees[e]}_{days[d]}_{shifts[s]}_{activities[a]}")

    # -------------------------
    # Contraintes
    # -------------------------

    # 1️⃣ Chaque créneau : 2 Téléphones, 1 Dérogation, 1 Réclamation
    for d in range(n_days):
        for s in range(n_shifts):
            # Chaque activité a le bon nombre de personnes
            model.Add(sum(x[e, d, s, 0] for e in range(n_emp)) == 2)  # Téléphone
            model.Add(sum(x[e, d, s, 1] for e in range(n_emp)) == 1)  # Dérogation
            model.Add(sum(x[e, d, s, 2] for e in range(n_emp)) == 1)  # Réclamation

            # Chaque employé fait exactement 1 activité par créneau
            for e in range(n_emp):
                model.Add(sum(x[e, d, s, a] for a in range(len(activities))) == 1)

    # 2️⃣ Pas plus de 3 jours de Téléphone par semaine par employé
    # Un jour est compté Téléphone s'il fait téléphone matin ou après-midi
    for e in range(n_emp):
        is_tel_day = []
        for d in range(n_days):
            tel_day = model.NewBoolVar(f"tel_day_{employees[e]}_{days[d]}")
            model.Add(sum(x[e, d, s, 0] for s in range(n_shifts)) >= 1).OnlyEnforceIf(tel_day)
            model.Add(sum(x[e, d, s, 0] for s in range(n_shifts)) == 0).OnlyEnforceIf(tel_day.Not())
            is_tel_day.append(tel_day)
        model.Add(sum(is_tel_day) <= 3)

    # 3️⃣ Pas 3 jours consécutifs de Téléphone
    for e in range(n_emp):
        for d in range(n_days - 2):
            model.Add(sum(x[e, d + i, s, 0] for i in range(3) for s in range(n_shifts)) <= 4)
            # (au max 4 créneaux sur 6 possibles pour 3 jours → évite 3 jours pleins de téléphone)

    # 4️⃣ Pas 2 jours consécutifs de Dérogation
    for e in range(n_emp):
        for d in range(n_days - 1):
            # Jour d : il fait dérogation ?
            derog_d = model.NewBoolVar(f"derog_day_{employees[e]}_{days[d]}")
            model.Add(sum(x[e, d, s, 1] for s in range(n_shifts)) >= 1).OnlyEnforceIf(derog_d)
            model.Add(sum(x[e, d, s, 1] for s in range(n_shifts)) == 0).OnlyEnforceIf(derog_d.Not())

            # Jour suivant : dérogation ?
            derog_next = model.NewBoolVar(f"derog_day_{employees[e]}_{days[d+1]}")
            model.Add(sum(x[e, d+1, s, 1] for s in range(n_shifts)) >= 1).OnlyEnforceIf(derog_next)
            model.Add(sum(x[e, d+1, s, 1] for s in range(n_shifts)) == 0).OnlyEnforceIf(derog_next.Not())

            # Pas deux jours consécutifs de dérogation
            model.AddBoolOr([derog_d.Not(), derog_next.Not()])

    # 5️⃣ Pas 2 jours consécutifs de Réclamation
    for e in range(n_emp):
        for d in range(n_days - 1):
            # Jour d : fait-il réclamation ?
            recl_d = model.NewBoolVar(f"reclam_day_{employees[e]}_{days[d]}")
            model.Add(sum(x[e, d, s, 2] for s in range(n_shifts)) >= 1).OnlyEnforceIf(recl_d)
            model.Add(sum(x[e, d, s, 2] for s in range(n_shifts)) == 0).OnlyEnforceIf(recl_d.Not())

            # Jour suivant : fait-il réclamation ?
            recl_next = model.NewBoolVar(f"reclam_day_{employees[e]}_{days[d+1]}")
            model.Add(sum(x[e, d+1, s, 2] for s in range(n_shifts)) >= 1).OnlyEnforceIf(recl_next)
            model.Add(sum(x[e, d+1, s, 2] for s in range(n_shifts)) == 0).OnlyEnforceIf(recl_next.Not())

            # Pas deux jours consécutifs de réclamation
            model.AddBoolOr([recl_d.Not(), recl_next.Not()])

    # 6️⃣ Objectif : minimiser les changements d'activité entre matin et après-midi
    penalties = []
    for e in range(n_emp):
        for d in range(n_days):
            # Variable binaire : 1 si les activités matin et après-midi sont différentes
            different_activity = model.NewBoolVar(f"different_{employees[e]}_{days[d]}")

            # Pour chaque activité, on vérifie si le matin et l'après-midi correspondent
            same_activity_bools = []
            for a in range(len(activities)):
                both_same = model.NewBoolVar(f"same_{employees[e]}_{days[d]}_{activities[a]}")
                model.AddBoolAnd([x[e, d, 0, a], x[e, d, 1, a]]).OnlyEnforceIf(both_same)
                model.AddBoolOr([x[e, d, 0, a].Not(), x[e, d, 1, a].Not()]).OnlyEnforceIf(both_same.Not())
                same_activity_bools.append(both_same)

            # Si au moins une activité est identique sur la journée, alors different_activity = 0
            model.AddBoolOr(same_activity_bools).OnlyEnforceIf(different_activity.Not())
            # Sinon, different_activity = 1
            model.AddBoolAnd([b.Not() for b in same_activity_bools]).OnlyEnforceIf(different_activity)

            penalties.append(different_activity)

    # On minimise le nombre total de changements d'activité sur la semaine
    model.Minimize(sum(penalties))
    
    # -------------------------
    # Solveur
    # -------------------------
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10
    solver.parameters.num_search_workers = 8

    result = solver.Solve(model)

    # -------------------------
    # Affichage du planning
    # -------------------------
    if result == cp_model.OPTIMAL or result == cp_model.FEASIBLE:
        for e in range(n_emp):
            st = f"{employees[e]}  →  "
            for d in range(n_days):
                for s in range(n_shifts):
                    for a in range(len(activities)):
                        if solver.Value(x[e, d, s, a]) == 1:
                            st += f"{activities[a]:6} "
            print(st)
    else:
        print("❌ Aucune solution trouvée.")

    # Statistics.
    print("\nStatistics")
    print(f"  - conflicts      : {solver.num_conflicts}")
    print(f"  - branches       : {solver.num_branches}")
    print(f"  - wall time      : {solver.wall_time} s")

if __name__ == "__main__":
    main()


A  →  Récla  Récla  Dérog  Dérog  Récla  Récla  Tél    Tél    Tél    Tél    
B  →  Tél    Tél    Récla  Récla  Tél    Tél    Tél    Tél    Dérog  Dérog  
C  →  Dérog  Dérog  Tél    Tél    Tél    Tél    Dérog  Dérog  Récla  Récla  
D  →  Tél    Tél    Tél    Tél    Dérog  Dérog  Récla  Récla  Tél    Tél    

Statistics
  - conflicts      : 0
  - branches       : 310
  - wall time      : 0.0237902 s


###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Second Test : Real situation
    </h3>
</div>

#### **Constraints**

1. An employee must have exactly one activity per shift

2. Number of employees per activity
- Minimum number of Tél >= 5
- Minimum number of Rens >= 3
- Minimum number of Dérog >= 1
- Minimum number of Récla == 1
- Minimum number of Imp == 1
- Minimum number of Libre <= 1

3. Employee cannot work more than 2 days consecutively on "Tél" activity

4. 


In [179]:
from ortools.sat.python import cp_model

class PartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, employees, days, shifts, activities, tasks, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._employees = employees
        self._days = days
        self._shifts = shifts
        self._activities = activities
        self._tasks = tasks
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        if len(self._shifts) == 2:
            print("          " + "           ".join(self._employees))
        else:
            print("       " + "     ".join(self._employees))
        for d in self._days:
            st = f"{d + 1:2} → "
            for e in self._employees:
                for s in self._shifts:
                    is_working = False
                    for a in self._activities:
                        if self.value(self._tasks[(e, d, s, a)]):
                            st += f"{a:5} "
                            is_working = True
                    if not is_working:
                        st += "  *   "
            print(st)
            if (d + 1) % 5 == 0:
                print()
        print()

        if self._solution_count >= self._solution_limit:
            print(f"\nStop search after {self._solution_limit} solutions")
            self.stop_search()

    def solutionCount(self):
        return self._solution_count


# Data.
n_employees = 14
employees = [chr(i) for i in range(65, 65 + n_employees)]
days = range(10)
shifts = ["Matin", "Après-midi"]
shifts = ["Matin"]
activities = ["Tél", "Rens", "Dérog", "Récla", "Imp", "Libre"]

# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# tasks[(e, d, s, a)]: employee 'e' works day 'd' on shift 's' on activity 'a'.
tasks = {}
for e in employees:
    for d in days:
        for s in shifts:
            for a in activities:
                tasks[(e, d, s, a)] = model.new_bool_var(f"task_{e}_{d}_{s}_{a}")


# ------------------------
# CONSTRAINTS

# Days Off
is_present = {item : [True] * len(days) * len(shifts) for item in employees}
# Alice does not want to work the first day
is_present['A'][1] = False
is_present['E'][5] = False
is_present['C'][8] = False

# 1️⃣ Each employee must have exactly one activity per shift.
for e in employees:
    for d in days:
        if is_present[e][d]:
            for s in shifts:
                model.add(sum(tasks[(e, d, s, a)] for a in activities) == 1)
        else:
            for s in shifts:
                model.add(sum(tasks[(e, d, s, a)] for a in activities) == 0)
                # model.Add(sum(tasks[(e, d, s, a)] for a in range(len(activities))) == 0)

# 2️⃣ Number of employees per activity.
for d in days:
    for s in shifts:
        # Minimum number of Tél >= 5
        model.add(sum(tasks[(e, d, s, activities[0])] for e in employees) >= 5)
        # Minimum number of Rens >= 3
        model.add(sum(tasks[(e, d, s, activities[1])] for e in employees) >= 3)
        # Minimum number of Dérog >= 1
        model.add_at_least_one(tasks[(e, d, s, activities[2])] for e in employees)
        # Minimum number of Récla == 1
        model.add_exactly_one(tasks[(e, d, s, activities[3])] for e in employees)
        # model.Add(sum(tasks[(e, d, s, activities[3])] for e in employees) == 1)
        # Minimum number of Imp == 1
        model.add_exactly_one(tasks[(e, d, s, activities[4])] for e in employees)
        # Minimum number of Libre <= 1
        model.add_exactly_one(tasks[(e, d, s, activities[5])] for e in employees)

# 3️⃣ Employee cannot work more than 2 days consecutively on "Tél" activity.
for e in employees:
    for d in range(len(days) - 2):
        model.add(sum(tasks[(e, d + i, s, activities[0])] for i in range(3) for s in shifts) <= 4)
        # (au max 4 créneaux sur 6 possibles pour 3 jours → évite 3 jours pleins de téléphone)

# 4️⃣ Same activity on morning and afternoon shifts (To Study)
# for e in employees:
#     for d in days:
#         # Variable binaire : 1 si les activités matin et après-midi sont différentes
#         different_activity = model.NewBoolVar(f"different_{e}_{d}")

#         # Pour chaque activité, on vérifie si le matin et l'après-midi correspondent
#         same_activity_bools = []
#         for a in activities:
#             both_same = model.NewBoolVar(f"same_{e}_{d}_{a}")
#             model.AddBoolAnd([tasks[(e, d, shifts[0], a)], tasks[(e, d, shifts[1], a)]]).OnlyEnforceIf(both_same)
#             model.AddBoolOr([tasks[(e, d, shifts[0], a)].Not(), tasks[(e, d, shifts[1], a)].Not()]).OnlyEnforceIf(both_same.Not())
#             same_activity_bools.append(both_same)

#         # Si au moins une activité est identique sur la journée, alors different_activity = 0
#         model.AddBoolOr(same_activity_bools).OnlyEnforceIf(different_activity.Not())
#         # Sinon, different_activity = 1
#         model.AddBoolAnd([b.Not() for b in same_activity_bools]).OnlyEnforceIf(different_activity)

#         # On peut ajouter une pénalité si on veut minimiser les changements d'activité
#         # model.Minimize(different_activity)

# 5️⃣ Constraints on activities (To Study)
# for e in employees:
#     for d in days:
#         for s in shifts:
#             # If employee e works on "Dérog" on day d and shift s, then they cannot work on "Dérog" the next day
#             if d < len(days) - 1:
#                 model.add_implication(tasks[(e, d, s, activities[2])], tasks[(e, d + 1, s, activities[2])].Not())
#             # If employee e works on "Récla" on day d and shift s, then they cannot work on "Récla" the next day
#             if d < len(days) - 1:
#                 model.add_implication(tasks[(e, d, s, activities[3])], tasks[(e, d + 1, s, activities[3])].Not())


# 6️⃣ 7️⃣ 8️⃣ 9️⃣ 🔟
# Days off
# for a in activities:
#     model.add(tasks[(employees[0], days[0], shifts[0], a)] == 0)
# ToDo

# Creates the solver and solve.
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

# Display the first five solutions.
solution_limit = 1
solution_printer = PartialSolutionPrinter(employees, days, shifts, activities, tasks, solution_limit)
status = solver.solve(model, solution_printer)

if status == cp_model.INFEASIBLE:
    print("Pas de solution !")

Solution 1
       A     B     C     D     E     F     G     H     I     J     K     L     M     N
 1 → Libre Imp   Rens  Rens  Récla Tél   Tél   Tél   Dérog Tél   Tél   Tél   Tél   Rens  
 2 →   *   Libre Rens  Rens  Tél   Dérog Imp   Tél   Tél   Tél   Récla Rens  Tél   Tél   
 3 → Dérog Récla Libre Rens  Rens  Imp   Tél   Tél   Tél   Tél   Tél   Tél   Tél   Rens  
 4 → Libre Rens  Récla Imp   Tél   Tél   Tél   Tél   Tél   Tél   Dérog Rens  Rens  Tél   
 5 → Rens  Imp   Dérog Libre Rens  Rens  Tél   Tél   Tél   Tél   Tél   Tél   Tél   Récla 

 6 → Rens  Imp   Dérog Rens    *   Tél   Tél   Tél   Tél   Tél   Récla Tél   Rens  Libre 
 7 → Libre Rens  Rens  Rens  Tél   Tél   Récla Tél   Tél   Tél   Tél   Dérog Imp   Tél   
 8 → Imp   Tél   Rens  Récla Tél   Libre Dérog Tél   Tél   Tél   Rens  Rens  Tél   Tél   
 9 → Rens  Imp     *   Rens  Dérog Récla Rens  Libre Tél   Tél   Tél   Tél   Tél   Tél   
10 → Récla Rens  Imp   Libre Rens  Rens  Tél   Tél   Tél   Tél   Tél   Tél   Tél   Dérog 



###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Third Test : Managing days off
    </h3>
</div>

In [171]:
from ortools.sat.python import cp_model

def main():
    # -------------------------
    # Données
    # -------------------------
    employees = ["A", "B", "C"]
    days = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
    activities = ["Telephone", "Reclamation", "Derogation"]

    # Présence (1 = présent, 0 = en congé)
    is_present = {
        "A": [1, 1, 1, 1, 1],
        "B": [1, 0, 1, 1, 1],
        "C": [1, 1, 1, 0, 0],
    }

    n_emp = len(employees)
    n_days = len(days)
    n_act = len(activities)

    # -------------------------
    # Modèle
    # -------------------------
    model = cp_model.CpModel()

    # x[e, d, a] = 1 si l'employé e fait l'activité a le jour d
    x = {}
    for e in range(n_emp):
        for d in range(n_days):
            for a in range(n_act):
                x[e, d, a] = model.NewBoolVar(f"x_{employees[e]}_{days[d]}_{activities[a]}")

    # -------------------------
    # Contraintes de présence / affectation
    # -------------------------
    for e, emp in enumerate(employees):
        for d in range(n_days):
            if is_present[emp][d] == 1:
                # Un seul poste si présent
                model.Add(sum(x[e, d, a] for a in range(n_act)) == 1)
            else:
                # Aucune activité si en congé
                model.Add(sum(x[e, d, a] for a in range(n_act)) == 0)

    # -------------------------
    # Couverture : uniquement si possible (suffisamment de présents)
    # -------------------------
    for d in range(n_days):
        # nombre de personnes présentes ce jour (constante connue)
        n_present = sum(is_present[emp][d] for emp in employees)

        # si on a au moins autant de présents que d'activités, on peut exiger >=1 par activité
        if n_present >= n_act:
            for a in range(n_act):
                model.Add(sum(x[e, d, a] for e in range(n_emp)) >= 1)
        else:
            # Option : ne rien imposer, ou imposer une règle alternative.
            # Ici on ne met rien : les présents seront affectés mais on n'exige pas
            # de couvrir toutes les activités quand il y a moins de présents.
            pass

    # -------------------------
    # Solveur
    # -------------------------
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 5
    result = solver.Solve(model)

    # -------------------------
    # Affichage
    # -------------------------
    if result == cp_model.OPTIMAL or result == cp_model.FEASIBLE:
        for d in range(n_days):
            print(f"\n=== {days[d]} ===")
            for e, emp in enumerate(employees):
                if is_present[emp][d] == 0:
                    print(f"  {emp} → CONGÉ")
                else:
                    for a in range(n_act):
                        if solver.Value(x[e, d, a]) == 1:
                            print(f"  {emp} → {activities[a]}")
    else:
        print("❌ Aucune solution trouvée")

if __name__ == "__main__":
    main()



=== Lundi ===
  A → Reclamation
  B → Telephone
  C → Derogation

=== Mardi ===
  A → Telephone
  B → CONGÉ
  C → Telephone

=== Mercredi ===
  A → Reclamation
  B → Telephone
  C → Derogation

=== Jeudi ===
  A → Telephone
  B → Telephone
  C → CONGÉ

=== Vendredi ===
  A → Telephone
  B → Telephone
  C → CONGÉ


In [ ]:
from ortools.sat.python import cp_model

class PartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, employees, days, shifts, activities, tasks, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._employees = employees
        self._days = days
        self._shifts = shifts
        self._activities = activities
        self._tasks = tasks
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        if len(self._shifts) == 2:
            print("          " + "           ".join(self._employees))
        else:
            print("       " + "     ".join(self._employees))
        for d in self._days:
            st = f"{d + 1:2} → "
            for e in self._employees:
                for s in self._shifts:
                    is_working = False
                    for a in self._activities:
                        if self.value(self._tasks[(e, d, s, a)]):
                            st += f"{a:5} "
                            is_working = True
                    if not is_working:
                        st += "  *   "
            print(st)
            if (d + 1) % 5 == 0:
                print()
        print()

        if self._solution_count >= self._solution_limit:
            print(f"\nStop search after {self._solution_limit} solutions")
            self.stop_search()

    def solutionCount(self):
        return self._solution_count


# Data.
n_employees = 14
employees = [chr(i) for i in range(65, 65 + n_employees)]
days = range(10)
shifts = ["Matin", "Après-midi"]
shifts = ["Matin"]
activities = ["Tél", "Rens", "Dérog", "Récla", "Imp", "Libre"]

# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# tasks[(e, d, s, a)]: employee 'e' works day 'd' on shift 's' on activity 'a'.
tasks = {}
for e in employees:
    for d in days:
        for s in shifts:
            for a in activities:
                tasks[(e, d, s, a)] = model.new_bool_var(f"task_{e}_{d}_{s}_{a}")


# ------------------------
# CONSTRAINTS

# Days Off
is_present = {item : [True] * len(days) * len(shifts) for item in employees}
# Alice does not want to work the first day
is_present['A'][1] = False
is_present['E'][5] = False
is_present['C'][8] = False

# 1️⃣ Each employee must have exactly one activity per shift.
for e in employees:
    for d in days:
        if is_present[e][d]:
            for s in shifts:
                model.add(sum(tasks[(e, d, s, a)] for a in activities) == 1)
        else:
            for s in shifts:
                model.add(sum(tasks[(e, d, s, a)] for a in activities) == 0)
                # model.Add(sum(tasks[(e, d, s, a)] for a in range(len(activities))) == 0)

# 2️⃣ Number of employees per activity.
for d in days:
    for s in shifts:
        # Minimum number of Tél >= 5
        model.add(sum(tasks[(e, d, s, activities[0])] for e in employees) >= 5)
        # Minimum number of Rens >= 3
        model.add(sum(tasks[(e, d, s, activities[1])] for e in employees) >= 3)
        # Minimum number of Dérog >= 1
        model.add_at_least_one(tasks[(e, d, s, activities[2])] for e in employees)
        # Minimum number of Récla == 1
        model.add_exactly_one(tasks[(e, d, s, activities[3])] for e in employees)
        # model.Add(sum(tasks[(e, d, s, activities[3])] for e in employees) == 1)
        # Minimum number of Imp == 1
        model.add_exactly_one(tasks[(e, d, s, activities[4])] for e in employees)
        # Minimum number of Libre <= 1
        model.add_exactly_one(tasks[(e, d, s, activities[5])] for e in employees)

# 3️⃣ Employee cannot work more than 2 days consecutively on "Tél" activity.
for e in employees:
    for d in range(len(days) - 2):
        model.add(sum(tasks[(e, d + i, s, activities[0])] for i in range(3) for s in shifts) <= 4)
        # (au max 4 créneaux sur 6 possibles pour 3 jours → évite 3 jours pleins de téléphone)

# 4️⃣ Same activity on morning and afternoon shifts (To Study)
# for e in employees:
#     for d in days:
#         # Variable binaire : 1 si les activités matin et après-midi sont différentes
#         different_activity = model.NewBoolVar(f"different_{e}_{d}")

#         # Pour chaque activité, on vérifie si le matin et l'après-midi correspondent
#         same_activity_bools = []
#         for a in activities:
#             both_same = model.NewBoolVar(f"same_{e}_{d}_{a}")
#             model.AddBoolAnd([tasks[(e, d, shifts[0], a)], tasks[(e, d, shifts[1], a)]]).OnlyEnforceIf(both_same)
#             model.AddBoolOr([tasks[(e, d, shifts[0], a)].Not(), tasks[(e, d, shifts[1], a)].Not()]).OnlyEnforceIf(both_same.Not())
#             same_activity_bools.append(both_same)

#         # Si au moins une activité est identique sur la journée, alors different_activity = 0
#         model.AddBoolOr(same_activity_bools).OnlyEnforceIf(different_activity.Not())
#         # Sinon, different_activity = 1
#         model.AddBoolAnd([b.Not() for b in same_activity_bools]).OnlyEnforceIf(different_activity)

#         # On peut ajouter une pénalité si on veut minimiser les changements d'activité
#         # model.Minimize(different_activity)

# 5️⃣ Constraints on activities (To Study)
# for e in employees:
#     for d in days:
#         for s in shifts:
#             # If employee e works on "Dérog" on day d and shift s, then they cannot work on "Dérog" the next day
#             if d < len(days) - 1:
#                 model.add_implication(tasks[(e, d, s, activities[2])], tasks[(e, d + 1, s, activities[2])].Not())
#             # If employee e works on "Récla" on day d and shift s, then they cannot work on "Récla" the next day
#             if d < len(days) - 1:
#                 model.add_implication(tasks[(e, d, s, activities[3])], tasks[(e, d + 1, s, activities[3])].Not())


# 6️⃣ 7️⃣ 8️⃣ 9️⃣ 🔟
# Days off
# for a in activities:
#     model.add(tasks[(employees[0], days[0], shifts[0], a)] == 0)
# ToDo

# Creates the solver and solve.
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

# Display the first five solutions.
solution_limit = 1
solution_printer = PartialSolutionPrinter(employees, days, shifts, activities, tasks, solution_limit)
status = solver.solve(model, solution_printer)

if status == cp_model.INFEASIBLE:
    print("Pas de solution !")